In [1]:
from enum import Enum
from math import cos, pi
import RPi.GPIO as GPIO          
from time import sleep
import picamera2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import copy
import os
os.environ["LIBCAMERA_LOG_LEVELS"] = "2"

# GPIO Configuration

![GPIO Layout](https://hackster.imgix.net/uploads/attachments/243430/FPJGHWDIWWFYSWW.LARGE.jpg?auto=compress%2Cformat&w=680&h=510&fit=max) 

You must correctly configure the GPIO pins connected to the DC motors, which you can change in the cell below. YOU need the GPIO numbers not the pin numbers and GPIO 21 is next to the camera connector on the Raspberry Pi.

You have to connect some free GPIO pins to the motor driver. You will need three pins per wheel, the PWM pin will control the speed, the other two together will control the direction. If the direction of a wheel is reversed, you can shwitch the pin numbers.


In [ ]:
RIGHT_PWM=22
RIGHT_1=17
RIGHT_2=27
LEFT_PWM=25
LEFT_1=23
LEFT_2=24

# Minimum speed

This is rather tricky to set. If you set this value too low, then setting a low speed on the car may not set the wheel in motion because it needs more energy to get started. If you set this value too high, the wheels cannot slow down to turn left/right. Good values are often somewhere between 40-60. The best value mostly depends on the power supply. Wheels running on 5V through a raspberry pi will not go as slow as wheels connected to higher voltages. Wheels with an external power source will likely work at 40, wheels power through the pi may need 50-60.

In [ ]:
MIN_SPEED=40

# Camera position

On some cars, the camera is mounted upside down. By setting this flag, the images are automatically flipped.

In [19]:
CAMERA_UPSIDE_DOWN = True

In [2]:
class Cam:
    """
    Singleton class to reuse the same camera instance. 
    You can get only one.
    """
    @classmethod
    def get(cls):
        try:
            return cls.cam
        except:
            cls.cam = picamera2.Picamera2()
            return cls.cam

In [3]:
class Direction(Enum):
    """
    An enum to define directions used by the Car class
    """
    FORWARD = 1
    BACKWARD = -1
    STOP = 0

    def reverse(self):
        if self == Direction.FORWARD:
            return Direction.BACKWARD
        elif self == Direction.BACKWARD:
            return Direction.FORWARD
        return Direction.STOP

In [4]:
class Wheel:
    """
    A single wheel on the car, that is controlled by three GPIO pins on the RPi.
    
    These wheels are designed to automatically update their speed based on the cars
    speed and angle. The angle will slowdown the speed of the wheel according to cosine(angle).
    angle=0 means full speed, angle=90 means stop and angle=-90 means full reverse.
    The intermediate speeds have not ben calibrated and therefore may not scale accordingly.

    Args:
      car: Car
        reference to the car to which this wheel is attached
      enable: int
        GPIO pin that is used for PWM (speed)
      in1, in2: int
        GPIO pins that are used to control direction
      minspeed, maxspeed: int (0-100)
        cab be used to control the minimum and maximum motor speed
    """
    def __init__(self, car, enable, in1, in2, minspeed=50, maxspeed=100):
        self.car = car
        self.enable = enable
        self.in1 = in1
        self.in2 = in2
        GPIO.setup(in1,GPIO.OUT)
        GPIO.setup(in2,GPIO.OUT)
        GPIO.setup(enable,GPIO.OUT)
        try:
            self.pwm = GPIO.PWM(self.enable, 2000)
        except:
            print(
'''
An exception occurred setting the PWM pin. 
You can only configure a pin for PWM once, therefore,
if you want to reinstantiate this object, you would
have to restart the notebook and shutdown other notebooks
using the same pin.
''')
            raise
        GPIO.output(self.in1,GPIO.LOW)
        GPIO.output(self.in2,GPIO.LOW)
        self._angle = 0
        self.minspeed=minspeed
        self.maxspeed=maxspeed
        self.speedrange=maxspeed-minspeed
        self.speed = 0
    
    @property
    def angle(self):
        """
        reflects the slowdown for this wheel
        """
        return self._angle

    @angle.setter
    def angle(self, value):
        assert value >= 0 and value <= 180, f'angle {value} for a wheel must be >= 0 and <= 180'
        if value != self.angle:
            self._angle = value
            self.update()

    def new_speed(self):
        """
        computes the modified speed of this wheel based on the speed of the car, direction and angle.
        """
        if self.car.direction == Direction.STOP or self.angle == 90:
            return 0
        elif self.car.direction == Direction.FORWARD and self.angle < 90:
            return self.minspeed + (cos(pi * self.angle / 180) * self.speedrange * self.car.speed) // 100
        else:
            return - (self.minspeed + (cos(pi * (360 - self.angle) / 180) * self.speedrange * self.car.speed) // 100)

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, value):
        """
        Sets the speed of a wheel directly.

        Args:
            value: int
                Should be a value between -100 and 100. Negative values move the wheel in reverse, 0 means stop.
        """
        self._speed = value
        if value == 0:
            GPIO.output(self.in1,GPIO.LOW)
            GPIO.output(self.in2,GPIO.LOW)    
            self.pwm.stop()
            self.pwmstarted = False
        elif not self.pwmstarted:
            self.pwm.start(value)
            self.pwmstarted = True
        else:
            self.pwm.ChangeDutyCycle(np.abs(value))
        if value > 0:
            GPIO.output(self.in1,GPIO.LOW)
            GPIO.output(self.in2,GPIO.HIGH)
        elif value < 0:
            GPIO.output(self.in1,GPIO.HIGH)
            GPIO.output(self.in2,GPIO.LOW)

    def update(self):
        """
        updates the wheel speed
        """
        self.speed = self.new_speed()

In [5]:
class Car:
    """
    A car that is controlled by this RPi, through direction, speed and angle properties.

    The car has two wheel that will modify their speed based on direction, speed and angle.
    """
    @classmethod
    def get(cls, camera_is_upside_down=CAMERA_UPSIDE_DOWN):
        """
        To allow grabbing several objects of a car, without complaints
        about the PWM pins being in use.
        """
        try:
            return cls._instance
        except:
            GPIO.setwarnings(False)
            GPIO.setmode(GPIO.BCM)
            self = cls()
            self._speed = 100
            self._direction = Direction.STOP
            self._angle = 0
            self.left = Wheel(self, LEFT_PWM, LEFT_1, LEFT_2, minspeed=MIN_SPEED)
            self.right = Wheel(self, RIGHT_PWM, RIGHT_1, RIGHT_2, minspeed=MIN_SPEED)
            self.camera_is_upside_down = camera_is_upside_down
            cls._instance = self
            return self
    
    @property
    def cam(self):
        try:
            return self._cam
        except:
            self._cam = Cam.get()
            return self._cam

    def new_image(self):
        """
        Take a new image
        """
        if not self.cam.started:
            self.cam.start()
        self._img = self.cam.capture_array()[:,:,:3]
        if self.camera_is_upside_down:
            self._img = self._img[::-1,::-1]
        return self._img

    def image(self):
        """
        Return the current image
        """
        try:
            return self._img
        except:
            return self.new_image()

    def save_image(self, folder, filename):
        """
        Save an image to a file

        Args:
            folder: Path
            filename: str
        """
        img = Image.fromarray(self.image())
        img.save( folder / filename ) 
    
    def show_image(self):
        """
        Shows an image in a notebook cell
        """
        plt.imshow(self.image())

    @property
    def direction(self):
        return self._direction

    @direction.setter
    def direction(self, value):
        if self._direction != value:
            self._direction = value
            self.left.update()
            self.right.update()
    
    @property
    def angle(self):
        return self._angle

    @angle.setter
    def angle(self, value):
        """
        controls the direction of the car by slowing down one of the wheels (-180 +180)
        angle = 0 is forward (although it will never go straight on 2 DC motors
        angle = 90 will stop the right wheel (turn around the right wheel)
        angle = 180 will reverse the right wheel (turn in place)
        angle = -90 will stop the left wheel (turn around the left wheel)
        angle = -180 will reverse the left wheel (turn in place)
        """
        if self.angle != value:
            self._angle = value
            if value > 0:
                self.right.angle = value
                self.left.angle = 0
            else:
                self.left.angle = -value
                self.right.angle = 0
    
    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, value):
        """
        controls the speed of the car, value 0-100
        """
        if self._speed != value:
            self._speed = value
            self.left.update()
            self.right.update()
        
    def forward(self):
        """
        Start the car moving forward, if the speed is big enough
        """
        self.direction = Direction.FORWARD

    def backward(self):
        """
        Start the car moving backwards.
        """
        self.direction = Direction.BACKWARD
        
    def stop(self):
        """
        Stop the car
        """
        self.direction = Direction.STOP

In [6]:
def demo_move():
    """
    Call this to get a demo of the car moving, make sure it has space.
    """
    c = Car.get()
    try:    # the try finally is here to make sure that if anything 
             # goes wrong, the car will stop
        c.speed = 100
        c.forward()
        sleep(1)
        c.angle=45
        sleep(1)
        c.angle=90
        sleep(1)
        c.angle=0
        sleep(1)
    finally:
        c.stop()

In [7]:
def demo_image():
    """
    Call this to grab and show an image from the camera
    """
    c = Car.get()
    c.show_image()